# Locus selection
Now let's have a closer look at the results of the reference-based assembly. It may very well be that many or most of your target loci did not receive good coverage accross all your samples. One useful thing to do is to make a selection of loci with good coverage and continue to only work with these loci for downstream analyses. `secapr` has a function that helps you to extract loci with good coverage.

In [1]:
%%bash
source activate secapr_env
secapr select_best_cov_loci -h

usage: secapr select_best_cov_loci [-h] --input INPUT --output OUTPUT [--n N]

Extract the n loci with the best read-coverage from you reference-based
assembly (bam-files)

optional arguments:
  -h, --help       show this help message and exit
  --input INPUT    The folder with the results of the reference based assembly
                   or the phasing results.
  --output OUTPUT  The output directory where results will be safed.
  --n N            The n loci that are best represented accross all samples
                   will be extracted.


This function will compile the average read-coverage for each locus and sample and will select the `n` loci with the best coverage accross all samples.
You can run this function simply like this:

    secapr select_best_cov_loci --input data/processed/remapped_reads --output data/processed/selected_loci --n 30

[Previous page](reference_assembly.ipynb) | [Next page](phasing.ipynb)

In [2]:
%%html
<iframe width="900" height="800" frameborder="0" scrolling="no" src="//plot.ly/~tobiashofmann/4.embed"></iframe>